# Preprocessing for Deep Learning NLP   

## 1. Introduction
在深度学习应用于自然语言处理（NLP）的背景下，数据预处理是一个至关重要的步骤，它通常包括以下几个环节：

1. **文本清洗（Text cleaning）**：
   - **去除噪声**：例如去除HTML标签、特殊字符等。
   - **规范化文本**：比如将所有文本转换为小写，以减少大小写造成的变体。
   - **分词（Tokenization）**：这是NLP中的一个基本步骤，涉及将连续的文本字符串分解为单独的单词或符号。
   - **去除停用词（Stop word removal）**：诸如“the”、“is”、“in”等单词通常对于理解文本的含义不是很有用，可以被移除。
   - **词干提取（Stemming）/词形还原（Lemmatization）**：比如将“running”还原为“run”，有助于模型理解不同词形的共同含义。

2. **构建词汇表（Vocabulary construction）**：
   - **确定模型的词汇**：基于训练数据创建一个单词集合。这一步将word映射到整数index，以便模型能够处理。

3. **词嵌入（Word embeddings）**：
   - **单词向量化**：使用预训练的词向量（如GloVe、Word2Vec）或在训练过程中学习词向量。这一步将已经创建好的词汇表中的单词映射到密集的向量表示。
   - 不同的词嵌入模型会学习到不同的语义信息，如词义、词性、语法关系等，有助于提高模型的性能。
4. **序列填充（Padding）/截断（Truncation）**：
   - **统一序列长度**：由于神经网络需要固定长度的输入，过长的序列可能需要截断，而过短的序列则需要通过填充额外的零或特定的占位符来扩展。

5. **构建标签（Label construction）**：
   - **准备监督学习标签**：对于有监督学习任务，比如分类或序列标注，需要准备相应的标签。

6. **数据增强（Data augmentation）**：
   - **扩展训练数据**：使用各种技术（如同义词替换、随机删除、句子重组）来创建更多的训练样本，有助于提高模型的鲁棒性和泛化能力。

7. **分批处理（Batching）**：
   - **准备用于训练的数据批次**：组织数据以便以批次的形式进行有效训练。

根据特定的任务和模型架构，预处理的具体步骤和实现可能会有所不同。例如，对于Transformer模型，通常会添加特殊的起始（[CLS]）和分隔（[SEP]）标记。而对于句子对任务，则可能需要确保两个句子合并后的长度不超过模型的最大序列长度。

In [2]:
import spacy
from datasets import load_dataset
from torchtext.data.utils import get_tokenizer
from collections import Counter
import torch
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.vocab import GloVe
from collections import Counter
from torch.utils.data import Dataset, DataLoader
from torchtext.vocab import vocab
from torch.nn import Embedding
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

## 2. Text Cleaning
spaCy是一个流行的自然语言处理库，提供了许多文本处理功能，包括分词、词形还原、命名实体识别等。在本节中，我们将使用spaCy来清理文本数据。

spaCy提供多种不同类型和大小的预训练模型，适用于多种语言。这些模型包括：

1. **核心模型（Core Models）**：
   - 包含了词性标注、句法分析、命名实体识别等功能。
   - 对于英语，提供了不同大小的模型，如`en_core_web_sm`（小型）、`en_core_web_md`（中型）、`en_core_web_lg`（大型），还有`en_core_web_trf`（基于transformer的模型）。

2. **多语种和语言特定模型**：
   - spaCy提供了对多种语言的支持，如西班牙语、法语、德语、俄语等。
   - 这些模型有不同的大小版本，一般也遵循`xx_core_web_sm`、`xx_core_web_md`、`xx_core_web_lg`的命名规范（`xx`是语言代码）。

3. **Transformer模型**：
   - 如`en_core_web_trf`，这类模型基于预训练的Transformer模型，如BERT或RoBERTa，它们通常提供更优的性能，尤其在理解上下文和复杂语义方面。

除了核心模型，spaCy社区还贡献了额外的模型和管道，这些可以通过spaCy官方网站和spaCy Universe来发现。这些额外的资源可能包括特定于某个领域的模型或用于特定任务的工具，比如情感分析、文本分类等。

模型的选择依赖于具体的任务和所需语言。一般而言，小型模型下载快、占用空间少，适合快速原型设计或当环境资源有限时使用；大型模型和基于Transformer的模型提供更高的精度和更好的性能，但下载和加载时间更长，需要更多的资源。

可以使用spaCy的命令行工具或在Python代码中使用`spacy.cli.download`函数来下载这些模型。使用时，需要确保选择的模型与你使用的spaCy版本兼容。

In [2]:
# 加载英语模型
nlp = spacy.load("en_core_web_sm")

# 处理文本
text = "Apple is looking at buying U.K. startup for $1 billion. The foxes walk into the box. The dog was barking."
doc = nlp(text)

# 打印文本
print(doc.text)

# 遍历文档中的句子
print('\nSentences:')
for sent in doc.sents:
    print(sent.text)
    
# 遍历文档中的命名实体
print('\nNamed Entities:')
for ent in doc.ents:
    print(ent.text, ent.label_)

# 打印每个token的文本和词性标注
print('\nTokens and their POS tags:')
for token in doc:
    print(token.text, token.pos_)

# 名词短语
print('\nNoun chunks:')
for chunk in doc.noun_chunks:
    print(chunk.text)
    
# 词形还原
print('\nLemmatized words:')
for token in doc:
    print(token.text, token.lemma_)


Apple is looking at buying U.K. startup for $1 billion. The foxes walk into the box. The dog was barking.

Sentences:
Apple is looking at buying U.K. startup for $1 billion.
The foxes walk into the box.
The dog was barking.

Named Entities:
Apple ORG
U.K. GPE
$1 billion MONEY

Tokens and their POS tags:
Apple PROPN
is AUX
looking VERB
at ADP
buying VERB
U.K. PROPN
startup NOUN
for ADP
$ SYM
1 NUM
billion NUM
. PUNCT
The DET
foxes NOUN
walk VERB
into ADP
the DET
box NOUN
. PUNCT
The DET
dog NOUN
was AUX
barking VERB
. PUNCT

Noun chunks:
Apple
U.K.
The foxes
the box
The dog

Lemmatized words:
Apple Apple
is be
looking look
at at
buying buy
U.K. U.K.
startup startup
for for
$ $
1 1
billion billion
. .
The the
foxes fox
walk walk
into into
the the
box box
. .
The the
dog dog
was be
barking bark
. .


In [2]:
# 加载本地的imdb数据集，arrow格式
data_files = {
    "train": './data_cache/imdb/plain_text/0.0.0/e6281661ce1c48d982bc483cf8a173c1bbeb5d31/imdb-train.arrow',
    "test": "./data_cache/imdb/plain_text/0.0.0/e6281661ce1c48d982bc483cf8a173c1bbeb5d31/imdb-test.arrow",
}
dataset = load_dataset('arrow', data_files=data_files)

# split the dataset,only use 1000 samples for demo
train_dataset = dataset['train'].select(range(1000)) # select 函数用于选择数据集的子集，这里选择前1000个样本
test_dataset = dataset['test'].select(range(500))

# 获取文本和标签
train_text = [sample["text"] for sample in train_dataset]
train_label = [sample["label"] for sample in train_dataset]
test_text = [sample["text"] for sample in test_dataset]
test_label =  [sample["label"] for sample in test_dataset]

# 将标签转换为Tensor
train_labels = torch.tensor(train_label, dtype=torch.long)
test_labels = torch.tensor(test_label, dtype=torch.long)

# 对于text数据，还需要预处理操作后再转换为Tensor

### 2.1 spaCy文本预处理
一般来说，spaCy的文本预处理包括以下几个步骤：
- 加载spaCy模型
- 定义文本预处理函数：
    - 使用`nlp.pipe`方法高效地处理大量文本,禁用不需要的组件以提高效率。
    - 去除停用词、标点和空格，并进行词形还原或其他处理。
    - 将处理后的文本重新组合成字符串。
- 对train_text和test_text进行预处理。

### nlp.pipe
nlp.pipe有着多种用法，其中最常见的用法是处理大量文本数据。在处理大量文本数据时，使用nlp.pipe方法可以提高处理效率，因为它会自动进行批处理，并在后台并行处理文本数据。此外，可以通过设置`disable`参数来禁用不需要的组件，以进一步提高处理速度。

向管道中添加组件
```
if not nlp.has_pipe("sentencizer"):
    nlp.add_pipe("sentencizer")
```

这里使用自定义预处理函数，对于每个文档，进行分词等步骤，返回结果train_text_clean和test_text_clean。格式为：[num_samples]，每个样本是一个列表，包含对应文档中的单词。

In [3]:
# 加载Spacy模型
nlp = spacy.load("en_core_web_sm")

# 检查nlp对象中是否已经有'sentencizer'组件，如果没有，则添加
if not nlp.has_pipe("sentencizer"):
    nlp.add_pipe("sentencizer")

# 定义文本预处理函数
# 该函数使用Spacy进行文本预处理，包括分词、词形还原和去除停用词
# 使用Spacy的管道（pipe）机制，可以高效地处理大量文本，优于直接使用nlp(text)方法；可选择禁用不需要的组件以提高效率
def preprocess_text(texts):
    tokenized_texts = []  # 存储处理后的文档列表

    for doc in nlp.pipe(texts, disable=["parser", "ner"]):
        # 去除停用词、标点和空格，并进行词形还原
        doc_tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
        tokenized_texts.append(doc_tokens)  # 将处理后的文档添加到结果中
    return tokenized_texts

train_text_clean = preprocess_text(train_text)
test_text_clean = preprocess_text(test_text)

In [86]:
# 打印预处理后的文本
print(train_text_clean[0])
len(train_text_clean) # 11196 表示训练集1000个样本，一共划分为11196个句子

['rent', 'curious', 'YELLOW', 'video', 'store', 'controversy', 'surround', 'release', '1967', 'hear', 'seize', 'U.S.', 'custom', 'try', 'enter', 'country', 'fan', 'film', 'consider', 'controversial', 'myself.<br', '/><br', '/>The', 'plot', 'center', 'young', 'swedish', 'drama', 'student', 'name', 'Lena', 'want', 'learn', 'life', 'particular', 'want', 'focus', 'attention', 'make', 'sort', 'documentary', 'average', 'swede', 'think', 'certain', 'political', 'issue', 'Vietnam', 'War', 'race', 'issue', 'United', 'States', 'ask', 'politician', 'ordinary', 'denizen', 'Stockholm', 'opinion', 'politic', 'sex', 'drama', 'teacher', 'classmate', 'marry', 'men.<br', '/><br', '/>What', 'kill', 'CURIOUS', 'YELLOW', '40', 'year', 'ago', 'consider', 'pornographic', 'sex', 'nudity', 'scene', 'far', 'shoot', 'like', 'cheaply', 'porno', 'countryman', 'mind', 'find', 'shocking', 'reality', 'sex', 'nudity', 'major', 'staple', 'swedish', 'cinema', 'Ingmar', 'Bergman', 'arguably', 'answer', 'good', 'old', 'bo

1000

## 3.Vecorization
文本向量化是将文本数据转换为数值形式的过程，以便计算机能够理解和处理。在自然语言处理中，文本向量化通常包括以下几种方法：
1. 稀疏表达：TDM（Term-Document Matrix）
    - Bag of Words（词袋模型）：将文本表示为词汇表中单词的出现次数。
    - TF-IDF（Term Frequency-Inverse Document Frequency）：将文本表示为单词的TF-IDF值，以衡量单词在文档中的重要性。
2. 密集表达：Word Embeddings
    - Word2Vec：将单词映射到低维空间的向量表示，以捕获单词之间的语义关系。
    - GloVe（Global Vectors for Word Representation）：基于全局词频统计的词向量。
    - FastText：Facebook提出的一种基于字符级n-gram的词向量方法。
    - BERT（Bidirectional Encoder Representations from Transformers）：基于Transformer的预训练模型，提供了上下文相关的词向量。

对于深度学习模型，通常使用词嵌入（Word Embeddings）来表示文本数据。在PyTorch中，可以使用`torch.nn.Embedding`层来加载预训练的词向量，或在训练过程中学习词向量。

### 3.1 Word2Vec词向量
Word2Vec是一种常用的词向量表示方法，它通过训练神经网络模型来学习单词的分布式表示。Word2Vec模型通常有两种架构：
- **Skip-gram**：通过给定中心词预测上下文词。
- **CBOW（Continuous Bag of Words）**：通过给定上下文词预测中心词。

参数说明：
- `tokenized_texts`：分词后的文本，作为训练数据输入。
- `sg=1`：使用Skip-Gram模型。如果设置为`0`，则使用CBOW模型。
- `min_count=1`：词语出现的最小次数。此参数确保了只有至少出现一次的词语才会被纳入训练。
- `window=3`：当前词与预测词在一个句子中的最大距离。
- `vector_size=100`：特征向量的维度大小。

Word2Vec模型的训练过程通常使用负采样（Negative Sampling）或层次Softmax（Hierarchical Softmax）来提高训练效率。Word2Vec模型的输出是每个单词的词向量，可以用于后续的文本分类、聚类、相似度计算等任务。

输出的数据格式为：[num_words, embedding_dim]，其中`num_words`是词汇表中单词的数量，`embedding_dim`是词向量的维度。

需要注意的是，Word2Vec是在单词级别上建立词向量的，这意味着它不会考虑到句子或文档的上下文信息。因此，如果你的任务是在文档级别进行情感分析（例如，给整个IMDB评论打标签），那么你可能需要采取一些策略来将单词级别的词向量转化为文档级别的表示。  一种常见的策略是对一个文档中所有单词的词向量取平均，得到一个代表整个文档的向量。然而，这种方法忽略了词序信息，可能会影响模型的性能。


In [4]:
from gensim.models import Word2Vec

# 使用全部文本数据训练Word2Vec模型
all_text_clean = train_text_clean + test_text_clean

# 训练Word2Vec模型，设置每个单词的向量维度为100
word2vec_model = Word2Vec(sentences=all_text_clean, vector_size=100, window=5, min_count=1, workers=4)

# 获取词向量
word_vectors = word2vec_model.wv

In [88]:
print(word_vectors['apple']) # 获取单词'apple'的词向量

[-2.2697045e-02  2.2419440e-02  6.9050072e-03 -3.5571195e-03
 -4.1548400e-03 -2.3723034e-02  1.2074275e-02  5.2477136e-02
 -3.1283759e-02 -1.8286215e-02 -9.5697008e-03 -3.8519885e-02
 -4.6982719e-03  2.4499275e-02  4.6533960e-04 -2.0590622e-02
  1.8333714e-02 -1.9812107e-02  1.7628982e-03 -3.9659768e-02
  2.1466682e-02  7.1030236e-03  1.0088837e-02 -1.4633660e-02
 -3.9604022e-03  3.1353601e-03 -1.6019225e-02  7.7307614e-04
 -3.6090154e-02  1.1287913e-02  3.4830797e-02  5.3286948e-03
  6.9953809e-03 -1.6698997e-02 -5.9348703e-03  1.5556945e-02
 -3.9932742e-03 -4.5184679e-03 -4.0730834e-03 -4.6856493e-02
  7.2372574e-03 -1.7406678e-02 -1.2130932e-02 -1.3593365e-02
  6.4295521e-03 -1.5891004e-02 -2.8445681e-03 -7.9976469e-03
  9.6949833e-03  1.2018349e-02 -1.7873716e-03 -2.9803008e-02
  6.6019587e-05 -7.7488702e-03  9.5908111e-04  5.2998197e-04
  1.1600737e-02 -3.6689639e-03 -1.8262729e-02 -4.6802030e-04
  1.9701929e-03 -1.1213666e-02 -9.2534320e-03 -9.6501652e-03
 -1.8676648e-02  2.90997

### 3.2 Padding and Truncation
建立好word2vec模型后，我们需要将文本数据转换为词向量表示，并进行填充或截断，以便输入到模型中。在PyTorch中，可以使用`torch.nn.utils.rnn.pad_sequence`函数来对序列进行填充。

下面代码会遍历每个文本，将文本中的每个词转换为词向量，然后将词向量序列填充或截断到指定的长度。
如果词不在训练好的词向量模型中，那么在当前的实现中，这个词的词向量将会是一个全零向量，因为我们使用`torch.zeros`初始化了词向量。

如果你希望对不在词向量模型中的词进行特殊处理，你可以选择以下几种策略：

1. 使用一个特定的向量来表示所有未知的词，比如随机向量。这可以通过在初始化向量时使用`torch.randn`代替`torch.zeros`来实现。

2. 使用一个特定的向量来表示所有未知的词，比如全一向量。这可以通过在初始化向量时使用`torch.ones`代替`torch.zeros`来实现。

3. 使用预训练词向量模型中的特殊向量，如"<UNK>"，来表示所有未知的词。这需要预训练词向量模型中包含这样的特殊向量。


In [89]:
import numpy as np

def document_vector(word2vec_model, doc):
    # 初始化一个全零向量
    doc_vector = np.zeros(word2vec_model.vector_size)
    num_words = 0
    for word in doc:
        # 如果单词在词汇表中，则加上其词向量
        if word in word2vec_model.wv:
            doc_vector += word2vec_model.wv[word]
            num_words += 1
    if num_words != 0:
        # 除以文档中词汇的总数，得到平均词向量
        doc_vector /= num_words
    return doc_vector

# 使用函数计算文档的平均词向量
train_doc_vectors = [document_vector(word2vec_model, doc) for doc in train_text_clean]
test_doc_vectors = [document_vector(word2vec_model, doc) for doc in test_text_clean]

In [90]:
len(train_doc_vectors)

1000

In [7]:
from torch.nn.utils.rnn import pad_sequence

# 定义文本-序列转换函数，该函数使用训练好的word2vec模型将文本转换为词向量序列
# 并使用pad_sequence函数对序列进行填充或截断
def text_to_sequence(texts, word_vectors, max_len):
    sequences = []
    for text in texts:
        seq = torch.zeros((len(text), word_vectors.vector_size))
        for i, word in enumerate(text):
            if i >= max_len: # 超过最大长度则跳出循环
                break
            if word in word_vectors: # 如果词在词向量中，则使用对应的词向量，否则使用全零向量
                seq[i] = torch.tensor(word_vectors[word])
                
        seq = seq[:max_len]  # 截断序列
        sequences.append(seq)
    return pad_sequence(sequences, batch_first=True, padding_value=0) # padding_value=0表示填充值为0;

max_len = 50  # 假定的最大序列长度
train_sequences = text_to_sequence(train_text_clean, word_vectors, max_len)
test_sequences = text_to_sequence(test_text_clean, word_vectors, max_len)

In [8]:
train_sequences.shape # torch.Size([1000, 50, 100])：1000个样本，每个样本的词向量序列长度为50，每个词向量的维度为100

torch.Size([1000, 50, 100])

### 3.3 DataLoader
自定义一个`Dataset`类，用于加载文本数据和标签数据，并定义一个`DataLoader`类，用于生成数据批次。`Dataset`类需要实现`__len__`和`__getitem__`方法，`DataLoader`类则可以指定批次大小、是否打乱数据等参数。

这一步最终得到train_dataloader和test_dataloader，可以直接用于模型的训练和测试。

train_dataloader.dataset会返回一个Dataset对象，每个元素是一个文本序列和对应的标签。train_dataloader.dataset.labels可以获取标签数据，train_dataloader.dataset.sequences可以获取文本数据。


In [11]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):
    def __init__(self, sequences, labels):
        self.sequences = sequences
        self.labels = labels

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return self.sequences[idx], self.labels[idx]

# 假设你已经有了对应的标签数据 train_labels 和 test_labels
train_dataset = TextDataset(train_sequences, train_labels)
test_dataset = TextDataset(test_sequences, test_labels)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

### 3.4 Doc2Vec
Doc2Vec是Word2Vec的扩展，它不仅学习单词的向量表示，还学习文档的向量表示。Doc2Vec模型通常有两种架构：
- **PV-DM**：通过给定上下文词和文档向量预测中心词。
- **PV-DBOW**：通过给定文档向量预测中心词。

Doc2Vec模型的训练过程与Word2Vec类似，但需要额外的文档标签。在训练过程中，文档标签可以作为额外的输入，以帮助模型学习文档的向量表示。

对于文档情感分析，如果使用word2vec模型，可以将文档中所有单词的词向量取平均，得到一个代表整个文档的向量。然而，这种方法忽略了词序信息，可能会影响模型的性能。
因此，使用Doc2Vec模型可以更好地捕获文档的语义信息，提高模型的性能。

In [28]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# 将文本数据转换为TaggedDocument对象
all_text_clean_tagged = [TaggedDocument(doc, [i]) for i, doc in enumerate(all_text_clean)]

# 训练Doc2Vec模型
doc2vec_model = Doc2Vec(vector_size=100, window=5, min_count=1, workers=4, epochs=20)
doc2vec_model.build_vocab(all_text_clean_tagged) # 构建词汇表
doc2vec_model.train(all_text_clean_tagged, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs) # 训练模型

# 获取文档向量,使用infer_vector方法，该方法可以对新文档进行向量化，输入为文档的分词列表，输出为文档的向量表示
# 获取训练集和测试集的文档向量
train_doc_vectors = [doc2vec_model.infer_vector(doc) for doc in train_text_clean]
test_doc_vectors = [doc2vec_model.infer_vector(doc) for doc in test_text_clean]

In [30]:
len(train_doc_vectors), len(test_doc_vectors)

(1000, 500)

#### GloVe词向量
GloVe是一种基于全局词频统计的词向量方法，它通过对词共现矩阵进行奇异值分解（SVD）来学习词向量。GloVe词向量通常在大型语料库上进行预训练，可以直接使用预训练的GloVe词向量，也可以在训练过程中学习词向量。

输出的数据格式为：[num_samples, embedding_dim]，其中`num_samples`是样本数量,即原数据的文档数量，`embedding_dim`是词向量的维度。

In [35]:
from torchtext.vocab import GloVe

# GloVe词向量，6B表示使用6亿词的预训练词向量，dim表示词向量的维度
glove = GloVe(name='6B', dim=100)

# 定义词嵌入函数，将文本转换为词向量
def text_to_embedding(texts, glove):
    embeddings = []
    for text in texts:
        tokens = text.split() # 分词
        embedding = [glove[token] for token in tokens if token in glove.stoi] # 获取词向量
        
        if len(embedding) > 0:
            embedding = torch.stack(embedding).mean(0)  # 取平均得到整个句子的嵌入表示
        else:
            # 如果句子中的所有词都不在词向量中，则使用全零向量表示
            embedding = torch.zeros(glove.vectors.shape[1])
        embeddings.append(embedding)
    return torch.stack(embeddings)

train_embeddings = text_to_embedding(train_text_clean, glove) # [1000, 100]: 1000个样本，每个样本的词向量维度为100
test_embeddings = text_to_embedding(test_text_clean, glove) # [500, 100]: 500个样本，每个样本的词向量维度为100

# 打印词嵌入
print(train_embeddings)

AttributeError: 'list' object has no attribute 'split'

## nn.Embedding
在PyTorch中，可以使用`torch.nn.Embedding`层来加载预训练的词向量，或在训练过程中学习词向量。`torch.nn.Embedding`层的输入是一个整数张量，表示单词的索引，输出是对应的词向量。

可以将文本数据直接转换为整数序列，并将这些整数序列作为模型的输入，而无需手动进行词嵌入。

1. 分词和预处理：对原始训练集进行分词和预处理，包括去除标点符号、停用词等，以及将文本转换为小写形式。

2. 构建词汇表：根据预处理后的训练集构建词汇表，将每个单词映射到一个唯一的整数。

3. 将文本转换为整数序列：将预处理后的文本数据转换为整数序列，其中每个单词都用其在词汇表中的整数表示。可以使用Python的字典或者torchtext库等工具来实现这一步骤。

4. 准备数据集：将转换后的整数序列组织成批次，并根据需要填充或截断成相同长度，以便输入到模型中。

5. 训练模型：将整数序列的批次作为模型的输入，直接进行训练。

通过这种方式，您可以直接将原始文本数据转换为整数序列，并将其传递给模型进行训练和测试，无需手动进行词嵌入。这种方法简化了数据准备过程，并使模型的使用更加方便。

In [47]:
from torch.nn.utils.rnn import pad_sequence
from torch import LongTensor

# 定义填充函数
def pad_sequences(sequences, padding_value=0):
    return pad_sequence([LongTensor(seq) for seq in sequences], batch_first=True, padding_value=padding_value)

# 修改你的函数，添加填充步骤
def text_to_int_sequence(texts, vocab):
    int_sequences = []
    for text in texts:
        int_sequence = [vocab[word] for word in text.split() if word in vocab]
        int_sequences.append(torch.LongTensor(int_sequence))  # 转换为长整型
    return pad_sequences(int_sequences)

# 重新转换文本为整数序列，并进行填充
train_int_sequences = text_to_int_sequence(train_text_clean, vocab)
test_int_sequences = text_to_int_sequence(test_text_clean, vocab)

# 打印转换和填充后的整数序列示例
print(train_int_sequences[0])
print(test_int_sequences[0])

tensor([ 222, 1061, 1845,  176,  723, 5177, 1200,  280, 7396,  184, 4060, 2003,
        2815,   33,  951,  604,   88,    3,  262, 1846, 7397,    1,   24,   17,
        2179,   93, 2816,  394,  370,  285, 1847,   30,  481,   46,  605,   30,
         620,  496,   35,  164,  952,  584, 5178,   12,  903, 1012,  518, 2180,
         332,  861,  518, 3339, 3340,  260, 2817, 2181, 7398, 4061,  621, 1695,
         142,  394,  585, 2818, 1013, 7399,    1,  827,   43, 1061, 1845,  724,
          41,  413,  262, 4062,  142,  467,   11,   85,  152,    4, 3341, 2467,
        7400,  158,   28, 1696,  468,  142,  467,  456, 5179, 2816,  265, 7401,
        3342, 4063, 1014,    6,   53,  165,  348, 2468,  142,   11, 5180,    1,
          69, 5181,  414,   76,  142,   79,    3,   79, 2004,  568, 1201,   22,
         103,   79, 4062,  371,  674, 1061, 1845,    6,    3,   30, 1697, 2819,
        4064, 2469, 1113, 2816,  265,    3,   17,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,   